In [1]:
import pandas as pd
df_train=pd.read_csv("train.csv")

In [2]:
# Separate features and target
X_train = df_train.drop(columns=["label"])
y_train = df_train["label"]

In [3]:
pip install bayesian-optimization


Note: you may need to restart the kernel to use updated packages.


In [4]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [5]:
def objective_bo(max_depth, n_estimators, min_samples_split, min_samples_leaf, criterion):
    criterion = 'gini' if criterion < 0.5 else 'entropy'
    clf = RandomForestClassifier(
        max_depth=int(max_depth),
        n_estimators=int(n_estimators),
        min_samples_split=int(min_samples_split),
        min_samples_leaf=int(min_samples_leaf),
        criterion=criterion,
        random_state=42
    )
    score = cross_val_score(
        clf, X_train, y_train,
        scoring='accuracy',
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    ).mean()
    return score

pbounds = {
    'max_depth': (5, 60),
    'n_estimators': (30, 100),
    'min_samples_split': (2, 10),
     'min_samples_leaf': (1, 5),
    'criterion': (0, 1)
}


In [6]:
optimizer = BayesianOptimization(f=objective_bo, pbounds=pbounds, random_state=42)
optimizer.maximize(init_points=30, n_iter=80)

print("Best parameters:", optimizer.max)

|   iter    |  target   | max_depth | n_esti... | min_sa... | min_sa... | criterion |
-------------------------------------------------------------------------------------
| 1         | 0.9396104 | 25.599706 | 96.550001 | 7.8559515 | 3.3946339 | 0.1560186 |
| 2         | 0.8765970 | 13.579698 | 34.065852 | 8.9294091 | 3.4044600 | 0.7080725 |
| 3         | 0.7346278 | 6.1321471 | 97.893689 | 8.6595411 | 1.8493564 | 0.1818249 |
| 4         | 0.8904384 | 15.087248 | 51.296957 | 6.1980514 | 2.7277800 | 0.2912291 |
| 5         | 0.9496364 | 38.651909 | 39.764570 | 4.3371571 | 2.4654473 | 0.4560699 |
| 6         | 0.9446368 | 48.184677 | 43.977164 | 6.1138755 | 3.3696582 | 0.0464504 |
| 7         | 0.9406876 | 38.414966 | 41.936688 | 2.5204127 | 4.7955421 | 0.9656320 |
| 8         | 0.9448046 | 49.461854 | 51.322963 | 2.7813769 | 3.7369321 | 0.4401524 |
| 9         | 0.8388176 | 11.712102 | 64.662383 | 2.2751081 | 4.6372816 | 0.2587799 |
| 10        | 0.9450769 | 41.438725 | 51.819775 | 6.16

In [7]:
best_rf_BOGP=optimizer.max

In [8]:
import joblib

In [9]:
# Convert numeric parameters to int
best_rf_BOGP = {k: int(v) if isinstance(v, (np.integer, np.floating)) else v for k, v in best_rf_BOGP.items()}

In [10]:
del best_rf_BOGP['target']

In [11]:
best_rf_BOGP=best_rf_BOGP['params']

In [12]:
# Convert numeric parameters to int
best_rf_BOGP = {k: int(v) if isinstance(v, (np.integer, np.floating)) else v for k, v in best_rf_BOGP.items()}

In [13]:
best_rf_BOGP['criterion']='gini'

In [14]:
joblib.dump(best_rf_BOGP, "best_rf_BOGP.pkl")

['best_rf_BOGP.pkl']